### ISyE524: Intro to Optimization
#### Problem Set #5
##### 1. Ahhhhhhhh, Daniel-son. You SuDoku Master
###### 1-1

\begin{align*}
\text{decision variables: } \\
\quad & X \in {0,1} ^{9 \times 9 \times 9} \\

\quad & X_{ijk} = 
  \begin{cases} 
   1 & \text{if } (i,j) \text{ entry is a } k \\
   0 & \text{otherwise} 
  \end{cases} \\

\text{constraints: } \\

\quad & \sum_{k=1}^{9} x_{ijk} = 1 \quad \forall i, j \\
& \sum_{i=1}^{9} x_{ijk} = 1 \quad \forall j, k \\
& \sum_{j=1}^{9} x_{ijk} = 1 \quad \forall i, k \\
& \sum_{(i, j) \in C } x_{ijk} = 1 \quad \forall C, k \\
\end{align*}

###### 1-2

In [1]:
# Given data. Unknown entries are specified as "0"
given = [
    0 6 0  1 0 4  0 5 0
    0 0 8  3 0 5  6 0 0
    2 0 0  0 0 0  0 0 1
 
    8 0 0  4 0 7  0 0 6
    0 0 6  0 0 0  3 0 0
    7 0 0  9 0 1  0 0 4
 
    5 0 0  0 0 0  0 0 2
    0 0 7  2 0 6  9 0 0
    0 4 0  5 0 8  0 7 0
  ];

In [3]:
# helper function to print a sudoku grid
function printSudoku(arr)
    u = 0
    println("+-------+-------+-------+")
    for p in 1:3:9
      for q in 0:2
        print("| ")
        for r in 1:3:9
          for s in 0:2
            u = round(Int, arr[p+q,r+s])
            u == 0 ? print(" ") : print(u)
            print(" ")
          end
          print("| ")
                  end
        println()
      end
      println("+-------+-------+-------+")
    end
  end
  ;

In [8]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[1:9,1:9,1:9], Bin)

# exactly one number per cell
for i in 1:9
  for j in 1:9
    @constraint(m, sum(x[i,j,k] for k in 1:9) == 1)
  end
end

# exactly one of each number per row
for i in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for j in 1:9) == 1)
  end
end

# exactly one of each number per column
for j in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for i in 1:9) == 1)
  end
end

# exactly one of each number per 3x3 block
for k in 1:9
  for p in 0:3:6
    for q in 0:3:6
      @constraint(m, sum(x[p+i,q+j,k] for i in 1:3, j in 1:3) == 1)
    end
  end
end

# initial conditions
for i in 1:9
  for j in 1:9
    if given[i,j] != 0
      @constraint(m, x[i,j,given[i,j]] == 1)
    end
  end
end

@time(optimize!(m))

# if termination_status(m) != :OPTIMAL
#     println(termination_status(m))
# else
    #generate solution grid and display the solution
    solution = zeros(9,9)
    for i in 1:9
      for j in 1:9
        for k in 1:9
          if value(x[i,j,k]) == 1
            solution[i,j] = k
            continue
          end
        end
      end
    end

    println("The solution is: ")
    printSudoku(solution)
# end

  0.009187 seconds (6.31 k allocations: 1.203 MiB)
The solution is: 
+-------+-------+-------+
| 9 6 3 | 1 7 4 | 2 5 8 | 
| 1 7 8 | 3 2 5 | 6 4 9 | 
| 2 5 4 | 6 8 9 | 7 3 1 | 
+-------+-------+-------+
| 8 2 1 | 4 3 7 | 5 9 6 | 
| 4 9 6 | 8 5 2 | 3 1 7 | 
| 7 3 5 | 9 6 1 | 8 2 4 | 
+-------+-------+-------+
| 5 8 9 | 7 1 3 | 4 6 2 | 
| 3 1 7 | 2 4 6 | 9 8 5 | 
| 6 4 2 | 5 9 8 | 1 7 3 | 
+-------+-------+-------+


###### 1-3

###### 1-4

###### 1-5

##### 2. Integer Programming is Smarter than a Fourth Grader?
###### 2-1

###### 2-2

##### 3. An Auror is Near
###### 3-1

###### 3-2

##### 4. Routing Apple Deliveries
###### 4-1

###### 4-2

##### 5. Paint Production
###### 5-1

###### 5-2